In [1]:
import os
import random
import shutil
import time
import warnings
#os.environ["CUDA_VISIBLE_DEVICES"] = "5"
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
# Data loading code

valdir = '/data/public/imagenet2012/val'
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

batch_size = 256
val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ])),
    batch_size=batch_size, shuffle=False,
    num_workers=4, pin_memory=True)

In [3]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
           
            images = images.cuda()
            target = target.cuda()

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % 10 == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'
    
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].contiguous().view(-1).float().sum(0, keepdim=True)  # remove .contiguous() in low version torch
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

## ACM 

In [4]:
import copy

def arnold(img, key):
    N, a, b, c, d = key  #  key=[1,1,1,1,2]
    h, w = img.shape[: 2]
    new_img = copy.deepcopy(img)

    for i in range(N):
        
        for x in range(h):
            
            for y in range(w):
                
                nx = ((a * x + b * y) % w + w) % w
                ny = ((c * x + d * y) % w + w) % w
                nx = int(nx)
                
                ny = int(ny)
                
                new_img[nx, ny] = img[x, y]
        img = copy.deepcopy(new_img)
    return new_img


def iarnold(img, key):
    N, a, b, c, d = key

    #matrix = np.mat([[a, b], [c, d]]).I

    [[a, b], [c, d]] = [[(a*b+1),-a],[-b,1]]
    return arnold(img, [N, a, b, c, d])

In [5]:
import torchvision.models as models
cudnn.benchmark = True
criterion = nn.CrossEntropyLoss().cuda()

# change this model
# model = models.googlenet(pretrained=True)
#model = models.resnet101(pretrained=True)
#model = models.mobilenet_v2(pretrained=True)
#model = models.densenet121(pretrained=True)
model = models.resnet18(pretrained=True)
#model = models.vgg16(pretrained=True)
model = model.cuda()

# name for each layer
for name, param in model.named_parameters():
    print(name,'    ', param.size())



# the original accuracy
validate(val_loader, model, criterion) 

conv1.weight      torch.Size([64, 3, 7, 7])
bn1.weight      torch.Size([64])
bn1.bias      torch.Size([64])
layer1.0.conv1.weight      torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight      torch.Size([64])
layer1.0.bn1.bias      torch.Size([64])
layer1.0.conv2.weight      torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight      torch.Size([64])
layer1.0.bn2.bias      torch.Size([64])
layer1.1.conv1.weight      torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight      torch.Size([64])
layer1.1.bn1.bias      torch.Size([64])
layer1.1.conv2.weight      torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight      torch.Size([64])
layer1.1.bn2.bias      torch.Size([64])
layer2.0.conv1.weight      torch.Size([128, 64, 3, 3])
layer2.0.bn1.weight      torch.Size([128])
layer2.0.bn1.bias      torch.Size([128])
layer2.0.conv2.weight      torch.Size([128, 128, 3, 3])
layer2.0.bn2.weight      torch.Size([128])
layer2.0.bn2.bias      torch.Size([128])
layer2.0.downsample.0.weight      torch.Size([128, 64, 1, 1])
layer2.

tensor(69.7580, device='cuda:0')

## Encryption

In [6]:
import time 
import numpy as np

#layer 1
weight = model.conv1.weight
weight_half = weight[:3,:3]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec'.format(total))

model.conv1.weight.data[:3,:3] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion) 

layer half weights size is:  torch.Size([3, 3, 7, 7])
need 0.000137 sec
Test: [  0/196]	Time  3.725 ( 3.725)	Loss 6.7134e-01 (6.7134e-01)	Acc@1  82.81 ( 82.81)	Acc@5  96.88 ( 96.88)
Test: [ 10/196]	Time  0.597 ( 0.876)	Loss 1.2217e+00 (9.0819e-01)	Acc@1  67.19 ( 77.31)	Acc@5  90.23 ( 92.65)
Test: [ 20/196]	Time  1.999 ( 0.870)	Loss 9.2844e-01 (9.3043e-01)	Acc@1  78.52 ( 76.38)	Acc@5  90.62 ( 92.52)
Test: [ 30/196]	Time  1.404 ( 0.830)	Loss 9.3338e-01 (8.9457e-01)	Acc@1  77.34 ( 77.36)	Acc@5  92.97 ( 92.74)
Test: [ 40/196]	Time  0.053 ( 0.778)	Loss 8.9877e-01 (9.3504e-01)	Acc@1  75.78 ( 75.75)	Acc@5  95.70 ( 92.93)
Test: [ 50/196]	Time  3.497 ( 0.856)	Loss 6.0718e-01 (9.2807e-01)	Acc@1  85.16 ( 75.61)	Acc@5  96.88 ( 93.24)
Test: [ 60/196]	Time  0.054 ( 0.867)	Loss 1.1745e+00 (9.3949e-01)	Acc@1  71.48 ( 75.34)	Acc@5  92.97 ( 93.33)
Test: [ 70/196]	Time  3.743 ( 0.907)	Loss 7.9589e-01 (9.2197e-01)	Acc@1  78.91 ( 75.86)	Acc@5  94.53 ( 93.43)
Test: [ 80/196]	Time  0.067 ( 0.879)	Loss 1.6319

tensor(69.2840, device='cuda:0')

In [7]:



import time 
import numpy as np

model = models.resnet18(pretrained=True)
model = model.cuda()


#layer 2
weight = model.layer1[0].conv1.weight
weight_half = weight[:64,:64]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec'.format(total))

model.layer1[0].conv1.weight.data[:64,:64] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion) 

layer half weights size is:  torch.Size([64, 64, 3, 3])
need 0.005261 sec
Test: [  0/196]	Time  3.464 ( 3.464)	Loss 9.7483e+00 (9.7483e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.39 (  0.39)
Test: [ 10/196]	Time  0.795 ( 0.942)	Loss 9.9751e+00 (1.0968e+01)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.21)
Test: [ 20/196]	Time  0.354 ( 0.887)	Loss 9.7799e+00 (1.0833e+01)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.13)
Test: [ 30/196]	Time  3.242 ( 0.980)	Loss 1.1808e+01 (1.0787e+01)	Acc@1   0.00 (  0.49)	Acc@5   0.00 (  0.98)
Test: [ 40/196]	Time  0.053 ( 0.920)	Loss 9.8546e+00 (1.0696e+01)	Acc@1   0.00 (  0.37)	Acc@5   0.00 (  0.74)
Test: [ 50/196]	Time  4.102 ( 0.922)	Loss 1.0644e+01 (1.0669e+01)	Acc@1   0.00 (  0.30)	Acc@5   0.00 (  0.60)
Test: [ 60/196]	Time  0.057 ( 0.895)	Loss 9.5025e+00 (1.0521e+01)	Acc@1   0.00 (  0.25)	Acc@5   0.00 (  0.50)
Test: [ 70/196]	Time  3.165 ( 0.922)	Loss 9.5753e+00 (1.0527e+01)	Acc@1   0.00 (  0.21)	Acc@5   0.00 (  0.43)
Test: [ 80/196]	Time  0.115 ( 0.917)	Loss 9.86

tensor(0.2160, device='cuda:0')

In [ ]:

import time 
import numpy as np

model = models.resnet18(pretrained=True)
model = model.cuda()


#layer 3
weight = model.layer1[0].conv2.weight
weight_half = weight[:64,:64]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec'.format(total))

model.layer1[0].conv2.weight.data[:64,:64] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion) 

layer half weights size is:  torch.Size([64, 64, 3, 3])
need 0.004942 sec
Test: [  0/196]	Time  4.175 ( 4.175)	Loss 6.5223e+00 (6.5223e+00)	Acc@1   3.52 (  3.52)	Acc@5  19.14 ( 19.14)
Test: [ 10/196]	Time  0.098 ( 1.044)	Loss 9.1497e+00 (1.0493e+01)	Acc@1   7.42 (  3.69)	Acc@5  22.66 ( 12.61)
Test: [ 20/196]	Time  1.206 ( 0.878)	Loss 1.6068e+01 (1.1047e+01)	Acc@1   0.39 (  2.90)	Acc@5   2.34 ( 10.42)
Test: [ 30/196]	Time  0.054 ( 0.834)	Loss 1.3400e+01 (1.1572e+01)	Acc@1   0.00 (  2.18)	Acc@5   0.78 (  8.03)
Test: [ 40/196]	Time  1.011 ( 0.825)	Loss 1.6469e+01 (1.2348e+01)	Acc@1   0.00 (  1.65)	Acc@5   0.00 (  6.11)
Test: [ 50/196]	Time  0.053 ( 0.836)	Loss 1.5083e+01 (1.2917e+01)	Acc@1   0.00 (  1.33)	Acc@5   0.00 (  4.92)
Test: [ 60/196]	Time  0.054 ( 0.822)	Loss 1.0111e+01 (1.3256e+01)	Acc@1   0.00 (  1.11)	Acc@5   3.12 (  4.24)
Test: [ 70/196]	Time  0.056 ( 0.843)	Loss 1.4570e+01 (1.3387e+01)	Acc@1   1.17 (  1.12)	Acc@5   2.34 (  4.20)


In [ ]:
import time 
import numpy as np

model = models.resnet18(pretrained=True)
model = model.cuda()


#layer 4
weight = model.layer1[1].conv1.weight
weight_half = weight[:64,:64]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec'.format(total))

model.layer1[1].conv1.weight.data[:64,:64] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)

In [ ]:
import time 
import numpy as np

model = models.resnet18(pretrained=True)
model = model.cuda()


#layer 5
weight = model.layer1[1].conv2.weight
weight_half = weight[:64,:64]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec'.format(total))

model.layer1[1].conv2.weight.data[:64,:64] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)

In [ ]:
#layer2.0.conv1.weight

import time 
import numpy as np

model = models.resnet18(pretrained=True)
model = model.cuda()


#layer 6
weight = model.layer2[0].conv1.weight
weight_half = weight[:64,:64]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec'.format(total))

model.layer2[0].conv1.weight.data[:64,:64] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)

In [ ]:
#layer2.0.conv2.weight

import time 
import numpy as np

model = models.resnet18(pretrained=True)
model = model.cuda()


#layer 7
weight = model.layer2[0].conv2.weight
weight_half = weight[:128,:128]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec'.format(total))

model.layer2[0].conv2.weight.data[:128,:128] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)

In [ ]:
#layer2.1.conv1.weight

import time 
import numpy as np

model = models.resnet18(pretrained=True)
model = model.cuda()


#layer 8
weight = model.layer2[1].conv1.weight
weight_half = weight[:128,:128]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec'.format(total))

model.layer2[1].conv1.weight.data[:128,:128] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)

In [ ]:

import time 
import numpy as np

model = models.resnet18(pretrained=True)
model = model.cuda()


#layer 9
weight = model.layer2[1].conv2.weight
weight_half = weight[:128,:128]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec'.format(total))

model.layer2[1].conv2.weight.data[:128,:128] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)

In [ ]:

import time 
import numpy as np

model = models.resnet18(pretrained=True)
model = model.cuda()


#layer 10
weight = model.layer3[0].conv1.weight
weight_half = weight[:128,:128]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec'.format(total))

model.layer3[0].conv1.weight.data[:128,:128] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)

In [ ]:
import time 
import numpy as np

model = models.resnet18(pretrained=True)
model = model.cuda()


#layer 11
weight = model.layer3[0].conv2.weight
weight_half = weight[:256,:256]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec'.format(total))

model.layer3[0].conv2.weight.data[:256,:256] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)

In [ ]:
import time 
import numpy as np

model = models.resnet18(pretrained=True)
model = model.cuda()


#layer 12
weight = model.layer3[1].conv1.weight
weight_half = weight[:256,:256]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec'.format(total))

model.layer3[1].conv1.weight.data[:256,:256] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)

In [ ]:
import time 
import numpy as np

model = models.resnet18(pretrained=True)
model = model.cuda()


#layer 13
weight = model.layer3[1].conv2.weight
weight_half = weight[:256,:256]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec'.format(total))

model.layer3[1].conv2.weight.data[:256,:256] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)

In [ ]:
import time 
import numpy as np

model = models.resnet18(pretrained=True)
model = model.cuda()


#layer 14
weight = model.layer4[0].conv1.weight
weight_half = weight[:256,:256]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec'.format(total))

model.layer4[0].conv1.weight.data[:256,:256] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)

In [ ]:
import time 
import numpy as np

model = models.resnet18(pretrained=True)
model = model.cuda()


#layer 15
weight = model.layer4[0].conv2.weight
weight_half = weight[:512,:512]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec'.format(total))

model.layer4[0].conv2.weight.data[:512,:512] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)

In [ ]:
import time 
import numpy as np

model = models.resnet18(pretrained=True)
model = model.cuda()


#layer 16
weight = model.layer4[1].conv1.weight
weight_half = weight[:512,:512]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec'.format(total))

model.layer4[1].conv1.weight.data[:512,:512] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)

In [ ]:
import time 
import numpy as np

model = models.resnet18(pretrained=True)
model = model.cuda()


#layer 17
weight = model.layer4[1].conv2.weight
weight_half = weight[:512,:512]
print('layer half weights size is: ',weight_half.size())
weight_numpy = weight_half.detach().cpu().numpy()
start = time.time()
arn_weight = arnold(weight_numpy,[1, 1,1,1,2])
end = time.time()
total = end - start
# print('the difference is',np.count_nonzero(weight_numpy-arn_weight))
print('need {:.6f} sec'.format(total))

model.layer4[1].conv2.weight.data[:512,:512] = torch.from_numpy(arn_weight).cuda()
#model.layer3[0].conv2.weight.data[:192,:192] = torch.from_numpy(arn_weight).cuda()
#model.conv2.conv.weight.data[:32,:32] = torch.from_numpy(arn_weight).cuda()
validate(val_loader, model, criterion)